In [4]:
import ccxt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from ta.trend import IchimokuIndicator
from ta.trend import ADXIndicator
from ta.trend import PSARIndicator

from ta.momentum import RSIIndicator
from ta.momentum import StochasticOscillator


from IPython.display import display
from tabulate import tabulate
from datetime import datetime

# Set the style of matplotlib to 'dark_background'
plt.style.use('dark_background')

# Configure pandas to display all rows and columns without line breaks
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)  # Set display width to 0 to prevent wrapping
pd.set_option('display.max_colwidth', None)  # Deprecated, use None instead


In [5]:
universe = [
    'BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'NEOUSDT', 'LTCUSDT', 'QTUMUSDT', 'ADAUSDT', 'XRPUSDT', 'EOSUSDT', 'IOTAUSDT',
    'XLMUSDT', 'ONTUSDT', 'TRXUSDT', 'ETCUSDT', 'ICXUSDT', 'VETUSDT', 'LINKUSDT', 'WAVESUSDT', 'HOTUSDT',
    'ZILUSDT', 'ZRXUSDT', 'FETUSDT', 'BATUSDT', 'XMRUSDT', 'ZECUSDT', 'IOSTUSDT', 'CELRUSDT', 'DASHUSDT', 'OMGUSDT',
    'THETAUSDT', 'ENJUSDT', 'MATICUSDT', 'ATOMUSDT', 'ONEUSDT', 'FTMUSDT', 'ALGOUSDT', 'DOGEUSDT', 'DUSKUSDT', 'ANKRUSDT',
    'COMBOUSDT', 'MTLUSDT', 'TOMOUSDT', 'DENTUSDT', 'KEYUSDT', 'CVCUSDT', 'CHZUSDT', 'BANDUSDT', 'XTZUSDT', 'RENUSDT',
    'RVNUSDT', 'HBARUSDT', 'NKNUSDT', 'STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT', 'RLCUSDT', 'BCHUSDT', 'FTTUSDT',
    'OGNUSDT', 'BTSUSDT', 'BNTUSDT', 'COTIUSDT', 'SOLUSDT', 'CTSIUSDT', 'CHRUSDT', 'MDTUSDT', 'STMXUSDT', 'KNCUSDT',
    'LRCUSDT', 'COMPUSDT', 'SCUSDT', 'ZENUSDT', 'SNXUSDT', 'DGBUSDT', 'SXPUSDT', 'MKRUSDT', 'STORJUSDT', 'MANAUSDT',
    'YFIUSDT', 'BALUSDT', 'BLZUSDT', 'SRMUSDT', 'ANTUSDT', 'CRVUSDT', 'SANDUSDT', 'OCEANUSDT', 'NMRUSDT', 'DOTUSDT',
    'RSRUSDT', 'TRBUSDT', 'SUSHIUSDT', 'KSMUSDT', 'EGLDUSDT', 'RUNEUSDT', 'UMAUSDT', 'BELUSDT', 'UNIUSDT', 'OXTUSDT',
    'AVAXUSDT', 'HNTUSDT', 'FLMUSDT', 'XVSUSDT', 'ALPHAUSDT', 'AAVEUSDT', 'NEARUSDT', 'FILUSDT', 'INJUSDT', 'AUDIOUSDT',
    'CTKUSDT', 'AXSUSDT', 'UNFIUSDT', 'ROSEUSDT', 'XEMUSDT', 'SKLUSDT', 'GRTUSDT', '1INCHUSDT', 'REEFUSDT', 'CELOUSDT',
    'TRUUSDT', 'CKBUSDT', 'LITUSDT', 'SFPUSDT', 'ALICEUSDT', 'LINAUSDT', 'PERPUSDT', 'CFXUSDT', 'TLMUSDT', 'BAKEUSDT',
    'ICPUSDT', 'ARUSDT', 'MASKUSDT', 'LPTUSDT', 'XVGUSDT', 'ATAUSDT', 'GTCUSDT', 'KLAYUSDT', 'C98USDT', 'QNTUSDT',
    'FLOWUSDT', 'MINAUSDT', 'RAYUSDT', 'DYDXUSDT', 'IDEXUSDT', 'GALAUSDT', 'YGGUSDT', 'AGLDUSDT', 'RADUSDT', 'DARUSDT',
    'BNXUSDT', 'ENSUSDT', 'JASMYUSDT', 'RNDRUSDT', 'FXSUSDT', 'HIGHUSDT', 'CVXUSDT', 'PEOPLEUSDT', 'SPELLUSDT', 'JOEUSDT',
    'ACHUSDT', 'IMXUSDT', 'API3USDT', 'WOOUSDT', 'TUSDT', 'ASTRUSDT', 'GMTUSDT', 'APEUSDT', 'GALUSDT', 'LDOUSDT',
    'OPUSDT', 'LEVERUSDT', 'STGUSDT', 'GMXUSDT', 'APTUSDT', 'HFTUSDT'
]

In [6]:
exchange = ccxt.binance({
    'enableRateLimit': True,
})

# Function to fetch OHLCV data
def fetch_ohlcv_data(symbol, timeframe):
    since = 0  # Set to 0 to fetch the full history
    all_ohlcv_data = []
    while True:
        ohlcv_data = exchange.fetch_ohlcv(symbol, timeframe, since)
        if not ohlcv_data:
            break
        since = ohlcv_data[-1][0] + 1  # Update since to the timestamp of the last candle
        all_ohlcv_data.extend(ohlcv_data)
    for i in range(len(all_ohlcv_data)):
        all_ohlcv_data[i][0] = datetime.utcfromtimestamp(all_ohlcv_data[i][0] / 1000)
    return all_ohlcv_data

ohlcv_daily = {}
start_dates = {}  # Dictionary to store the start dates for each symbol

# Fetch and store the full history of daily OHLCV data for each symbol
for symbol in universe:
    daily_data = fetch_ohlcv_data(symbol, '1h')
    ohlcv_daily[symbol] = pd.DataFrame(daily_data, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    
    # Calculate Stochastic Oscillator
    stochastic_oscillator = StochasticOscillator(high=ohlcv_daily[symbol]['high'], low=ohlcv_daily[symbol]['low'], close=ohlcv_daily[symbol]['close'], window=14, smooth_window=3)
    ohlcv_daily[symbol]['stoch_k'] = stochastic_oscillator.stoch()
    
    # RSI
    rsi_indicator = RSIIndicator(close=ohlcv_daily[symbol]['close'], window=14)
    ohlcv_daily[symbol]['rsi'] = rsi_indicator.rsi()
    
    # Store the start date for the symbol
    start_dates[symbol] = ohlcv_daily[symbol]['date'].iloc[0]

# Create a DataFrame from the start_dates dictionary
start_dates_df = pd.DataFrame(list(start_dates.items()), columns=['symbol', 'start_date'])

start_dates_df

,symbol,start_date
0,BTCUSDT,2017-08-17 04:00:00
1,ETHUSDT,2017-08-17 04:00:00
2,BNBUSDT,2017-11-06 03:00:00
3,NEOUSDT,2017-11-20 03:00:00
4,LTCUSDT,2017-12-13 03:00:00
5,QTUMUSDT,2018-03-19 08:00:00
6,ADAUSDT,2018-04-17 04:00:00
7,XRPUSDT,2018-05-04 08:00:00
8,EOSUSDT,2018-05-28 05:00:00
9,IOTAUSDT,2018-05-31 09:00:00


In [7]:
def fetch_full_ohlcv_history(symbol, timeframe='1d'):
    all_ohlcv = []
    since = 0  # Set to 0 to fetch from the earliest available data
    while True:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, 1000)
        if not ohlcv:
            break
        all_ohlcv.extend(ohlcv)
        since = ohlcv[-1][0] + 1  # Increment 'since' to get the next batch of data
    df = pd.DataFrame(all_ohlcv, columns=['date', 'open', 'high', 'low', 'close', 'volume'])  # Renamed 'timestamp' to 'date'
    df['date'] = pd.to_datetime(df['date'], unit='ms')
    df.set_index('date', inplace=True)
    return df

# Fetch the full history of daily prices for Bitcoin
btc_ohlcv = fetch_full_ohlcv_history('BTCUSDT')

# Calculate daily returns for Bitcoin
btc_ohlcv['returns'] = btc_ohlcv['close'].pct_change()

In [11]:
def is_btc_return_positive(date):
    return btc_ohlcv.loc[date, 'returns'] > 0

stoch_rsi_stats = {}

# Loop through each symbol in the ohlcv_daily dictionary
for symbol in ohlcv_daily:
    
    # Initialize counters for successful combined predictions
    successful_combined_predictions = 0
    combined_signal_count = 0
    percentage_increases = []
    
    # Loop through the data to find %K below 20 and RSI below 30
    for i in range(1, len(ohlcv_daily[symbol]) - 5):  # Exclude the last 10 days for forward checking
        
        # Check for %K below 20 and RSI below 30
        if ohlcv_daily[symbol]['stoch_k'][i] < 20 and ohlcv_daily[symbol]['rsi'][i] < 30:

            current_date = ohlcv_daily[symbol]['date'][i].strftime("%Y-%m-%d")
            btc_return_positive = is_btc_return_positive(current_date)

            if btc_return_positive: 
                combined_signal_count += 1
                
                # Check if the price increased after 5 periods
                if ohlcv_daily[symbol]['close'][i + 5] > ohlcv_daily[symbol]['close'][i]:
                    increase = (ohlcv_daily[symbol]['close'][i + 5] / ohlcv_daily[symbol]['close'][i]) / 100
                    percentage_increases.append(increase)
                    successful_combined_predictions += 1
    
    # Calculate the effectiveness percentage
    if combined_signal_count > 0:
        combined_effectiveness_percentage = (successful_combined_predictions / combined_signal_count) * 100
    else:
        combined_effectiveness_percentage = None

    mean_percent = np.mean(percentage_increases) 
    
    # Store the combined stats in the dictionary
    stoch_rsi_stats[symbol] = {'signal_count': combined_signal_count, 'effectiveness': combined_effectiveness_percentage, 'average_increase': mean_percent}

# Sort the dictionary by effectiveness in decreasing order
sorted_stoch_rsi_stats = sorted(stoch_rsi_stats.items(), key=lambda item: item[1]['effectiveness'] if item[1]['effectiveness'] is not None else -1, reverse=True)

# Display the sorted results
for symbol, stats in sorted_stoch_rsi_stats:
    if stats['effectiveness'] is not None:
        print(f"{symbol}: Combined Stochastic & RSI Effectiveness: {stats['effectiveness']:.2f}%, Signal Count: {stats['signal_count']} with average increase of {stats['average_increase']}")
    else:
        print(f"{symbol}: No combined Stochastic & RSI signals, Signal Count: {stats['signal_count']}")

BTCUSDT: Combined Stochastic & RSI Effectiveness: 86.02%, Signal Count: 236 with average increase of 0.010240529593615077
ALICEUSDT: Combined Stochastic & RSI Effectiveness: 84.78%, Signal Count: 184 with average increase of 0.010289142514348901
MANAUSDT: Combined Stochastic & RSI Effectiveness: 82.91%, Signal Count: 199 with average increase of 0.010295466174837408
XTZUSDT: Combined Stochastic & RSI Effectiveness: 81.57%, Signal Count: 217 with average increase of 0.010273971209137941
DGBUSDT: Combined Stochastic & RSI Effectiveness: 80.41%, Signal Count: 194 with average increase of 0.010291020297012916
STXUSDT: Combined Stochastic & RSI Effectiveness: 80.32%, Signal Count: 310 with average increase of 0.01030972885579936
EGLDUSDT: Combined Stochastic & RSI Effectiveness: 80.28%, Signal Count: 213 with average increase of 0.010245910551669579
ETCUSDT: Combined Stochastic & RSI Effectiveness: 80.18%, Signal Count: 227 with average increase of 0.010258641384432811
LTCUSDT: Combined Sto

: 